<a href="https://colab.research.google.com/github/JyothiLakshmiN/the-model-collection/blob/main/bert_qa_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install transformers torch --quiet


In [ ]:
# Import required modules
from transformers import BertForQuestionAnswering, BertTokenizer
import torch


In [ ]:
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
qa_dataset = [
    {
        "context": """
        Artificial Intelligence (AI) is the simulation of human intelligence processes by machines, especially computer systems.
        These processes include learning, reasoning, and self-correction. The term AI was first coined by John McCarthy in 1956
        at the Dartmouth Conference. Today, AI is widely used in applications such as natural language processing, computer vision,
        and autonomous vehicles.
        """,
        "questions": [
            "Who coined the term Artificial Intelligence?",
            "When was the term AI first introduced?",
            "Name one modern application of AI.",
            "What are the three main processes involved in AI?"
        ]
    },
    {
        "context": """
        The water cycle describes how water evaporates from the surface of the Earth, rises into the atmosphere, cools and condenses
        into rain or snow in clouds, and falls again to the surface as precipitation. The cycle is a continuous process that is vital
        for maintaining life on Earth. Solar energy drives the water cycle by heating the oceans and causing evaporation.
        """,
        "questions": [
            "What drives the water cycle?",
            "What happens to water when it rises into the atmosphere?",
            "Why is the water cycle important?",
            "Where does evaporation mainly occur?"
        ]
    },
    {
        "context": """
        The Internet is a global system of interconnected computer networks that use the standard Internet protocol suite (TCP/IP)
        to serve billions of users worldwide. It was originally developed by the U.S. Department of Defense in the 1960s under the
        project name ARPANET. Over time, it evolved to connect academic institutions and eventually the general public.
        """,
        "questions": [
            "Who developed the Internet?",
            "What protocol does the Internet use?",
            "What was the original name of the Internet project?",
            "When was the Internet first developed?"
        ]
    },
    {
        "context": """
        The human heart is a muscular organ that pumps blood throughout the body. It has four chambers: two atria and two ventricles.
        The right side of the heart receives deoxygenated blood and sends it to the lungs, while the left side pumps oxygenated blood
        to the rest of the body. The average human heart beats about 72 times per minute.
        """,
        "questions": [
            "How many chambers does the human heart have?",
            "What does the right side of the heart do?",
            "Where does the left side of the heart pump blood?",
            "What is the average heart rate per minute?"
        ]
    },
    {
        "context": """
        The Great Wall of China is one of the most famous landmarks in the world. It was built over several centuries to protect Chinese
        states and empires from invasions. The wall stretches over 13,000 miles and was constructed primarily during the Ming Dynasty.
        It is made of stone, brick, tamped earth, and wood.
        """,
        "questions": [
            "Why was the Great Wall of China built?",
            "How long is the Great Wall of China?",
            "During which dynasty was the wall primarily constructed?",
            "What materials were used to build the Great Wall?"
        ]
    },
    {
        "context": """
        The Sun is a nearly perfect sphere of hot plasma at the center of our solar system. It provides the energy necessary for life
        on Earth through light and heat. The Sun’s energy is produced by nuclear fusion reactions that convert hydrogen into helium.
        It has a surface temperature of about 5,500 degrees Celsius.
        """,
        "questions": [
            "What type of star is the Sun?",
            "What process produces the Sun’s energy?",
            "What elements are involved in the Sun’s fusion reaction?",
            "What is the surface temperature of the Sun?"
        ]
    }
]

In [ ]:
def get_answer(question, context):
    inputs = tokenizer.encode_plus(question, context, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    with torch.no_grad():
        outputs = model(**inputs)
        start_scores = outputs.start_logits
        end_scores = outputs.end_logits

    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores) + 1

    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end])
    )
    return answer


In [ ]:
for i, item in enumerate(qa_dataset, start=1):
    print(f"\n===============================")
    print(f"📘 Passage {i}")
    print(f"===============================")
    print(item["context"].strip())

    for q in item["questions"]:
        answer = get_answer(q, item["context"])
        print(f"\n❓ Question: {q}")
        print(f"✅ Answer: {answer}")

print("\n✅ All passages and questions processed successfully!")


📘 Passage 1
Artificial Intelligence (AI) is the simulation of human intelligence processes by machines, especially computer systems.
        These processes include learning, reasoning, and self-correction. The term AI was first coined by John McCarthy in 1956
        at the Dartmouth Conference. Today, AI is widely used in applications such as natural language processing, computer vision,
        and autonomous vehicles.

❓ Question: Who coined the term Artificial Intelligence?
✅ Answer: john mccarthy

❓ Question: When was the term AI first introduced?
✅ Answer: 1956

❓ Question: Name one modern application of AI.
✅ Answer: natural language processing , computer vision , and autonomous vehicles

❓ Question: What are the three main processes involved in AI?
✅ Answer: learning , reasoning , and self - correction

📘 Passage 2
The water cycle describes how water evaporates from the surface of the Earth, rises into the atmosphere, cools and condenses
        into rain or snow in clouds, a